<a href="https://colab.research.google.com/github/FZgabor/ML2025/blob/main/MLh%C3%A1zi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


Házi feladat

A Melbourne dataset felhasználásával készítsen egy lineáris regresszort a lakásárak (Price) meghatározására, úgy hogy a hiányzó adatokat (numerikus és kategórikus) imputálással pótolja (stratégiát Ön választ, lehetőleg adatot ne dobjon el, csak ). A modell építés során numerikus és kategórikus adatokat is használja fel! Az adatokat ossza fel 80% tréning, 20% validációs részre az imputálás után. A modellt értékelje ki MAE, R2 segítségével.

Folyamatok:

    adatok betöltése
    adatok felosztása tréning, teszt DataFramekre
    adatok (tréning, teszt) szétválasztása numerikus és kategórikus részekre
    adatok imputálása
    a kategórikus adatok kódolása
    numerikus és kategórikus adatok összefűzése (pd.concat([num_X_train, OH_cols_train], axis=1))
    modell készítése
    kiértékelés



In [2]:
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, r2_score


In [3]:
# Adatok betöltése
data = pd.read_csv(
    'https://raw.githubusercontent.com/karsarobert/MachineLearning2025/main/melb_data.csv'
)

# Célváltozó
y = data['Price']

# Feature-ök
X = data.drop(columns=['Price'])


In [4]:
numeric_features = X.select_dtypes(include=['int64', 'float64']).columns
categorical_features = X.select_dtypes(include=['object']).columns


In [5]:
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median'))
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])


In [6]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ]
)

model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', LinearRegression())
])


In [7]:
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42
)


In [8]:
model.fit(X_train, y_train)


Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='median'))]),
                                                  Index(['Unnamed: 0', 'Rooms', 'Distance', 'Postcode', 'Bedroom2', 'Bathroom',
       'Car', 'Landsize', 'BuildingArea', 'YearBuilt', 'Lattitude',
       'Longtitude', 'Propertycount'],
      dtype='object')),
                                                 ('cat',
                                                  Pipeline(steps=[('imputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehot',
                                                                   OneHotEncoder(handle_unknown='ignore'))]),
                                                  Index(['Suburb', 'Address', 'Type', 'Method', 'SellerG', 'Date', 'CouncilArea',
       'Regionname'],
      dtype='object'))])),
                ('regressor', LinearRegression())])

In [9]:
y_pred = model.predict(X_val)

mae = mean_absolute_error(y_val, y_pred)
r2 = r2_score(y_val, y_pred)

print(f"MAE: {mae:,.0f}")
print(f"R²: {r2:.3f}")


MAE: 245,371
R²: 0.661
